Just look to see if the sequences overlap

In [ ]:
x=2
x
import re

def peptide_equals(pep1, pep2):
    pep1 = re.sub('[IL]', '[IL]', pep1)
    return bool(re.match(pep1, pep2))

seq1 = 'HGLTVHK'
seq2 = 'HGLTVHK'
seq3 = 'HGITVHK'



print(peptide_equals(seq1, seq2))

print(peptide_equals(seq1, seq3))

print('Seq1: {}, Seq2: {}, Seq3: {}'.format(seq1, seq2, seq3))

In [ ]:
## Load any changes to local modules
%load_ext autoreload
%autoreload 2

import os
import sys

pwd = %pwd

module_path = os.path.abspath(os.path.join('{0}/../../'.format(pwd)))
if module_path not in sys.path:
    sys.path.append(module_path)

study_dir = '{}/{}'.format(module_path, 'results/03072018_proteomics_informatics_tc/')
results_dir = '{}/{}'.format(module_path, 'results/03072018_proteomics_informatics_tc/results')

#results_dir = '{}/{}'.format('/scratch/jonesmic', '03072018_proteomics_informatics_tc/results')



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import re
import os.path
import numpy as np
%matplotlib inline
from IPython.display import display
from scipy.stats import spearmanr, pearsonr

pd.set_option('precision', 5)
pd.set_option('max_colwidth', 50)

pd.options.display.float_format = '{:,.2f}'.format



from nbcpact import AnalyzeQuantCompare,Peptide,PeptideGroup,UcbreUtils,PeptidesFromPeptideListBuilder
from dpro.protdisc import PDReaderFactory


In [ ]:
class Comparison:
    data_cache = {}
    def __init__(self,
                 comparisonID=None,
                 nibr_input=None, 
                 ucb_input=None, 
                 ucb_output=None, 
                 desc=None, 
                 fdr=0.01):
        self.comparisonID = comparisonID
        self.nibr_input = nibr_input
        self.ucb_input = ucb_input
        self.ucb_output = ucb_output
        self.desc = desc.copy()
        self.Description = desc['Description']
        self.fdr = fdr
        
    def get_nibr_output(self):
        ## Check if the file exists
        file_path = '{}/{}'.format(results_dir, '{}_pd_data.txt'.format(comp.comparisonID))
        if not os.path.isfile(file_path):
            pdReader = self.get_pd_reader()
            df = pdReader.get_target_peptides()
            df = self.calc_pd_ratios(df)
            
            df.to_csv(file_path, sep='\t', index=False)
        
        df = pd.read_table(file_path)
        
        return df     
    
    def calc_pd_ratios(self, df):
        maxratio = 1000
        minratio  = 1/maxratio

        ratio = df['isoTOP TEV Light'] / df['isoTOP TEV Heavy']
        ratio = ratio.replace(np.inf, maxratio).replace(0, minratio)
        df['Light/Heavy R'] = ratio

        ratio = df['isoTOP TEV Heavy'] / df['isoTOP TEV Light'] 
        ratio = ratio.replace(np.inf, maxratio).replace(0, minratio)
        df['Heavy/Light R'] = ratio

        return df
        
        
    def get_pd_reader(self):
        if not self.nibr_input in self.data_cache.keys():
            if self.nibr_input:
                if os.path.isfile(self.nibr_input):
                    factory = PDReaderFactory()
                    pdReader = factory.createPDReader(self.nibr_input,
                                                      include_non_quant=True,
                                                      fdr=self.fdr,
                                                      validate=True, 
                                                      use_channel_clean_names=False)
                    self.data_cache[self.nibr_input] = pdReader

        if self.nibr_input in self.data_cache.keys():
            return self.data_cache[self.nibr_input]
        else: 
            return None
    
    def get_ucb_input(self):
        if not self.ucb_input in self.data_cache.keys():
            if self.ucb_input:
                if os.path.isfile(self.ucb_input):
                    df = self.load_ip2_peptide_list(self.ucb_input)
                    self.data_cache[self.ucb_input] = df

        return self.data_cache[self.ucb_input]
    
    
    def load_ip2_peptide_list(self, file_path):
        """
            Create a DF with these cols
            ['UNIQUE_1', 'annotation', 'area_ratios', 'decoy',
           'file_AREA_MEDIAN_RATIO_1', 'ip2_peptide', 'mod_locs', 'peptide',
           'ptm_indices', 'run_counter', 'run_data', 'uniprot_ids'],

        """
        peptidesFromPeptideListBuilder = PeptidesFromPeptideListBuilder(file_path)
        df = peptidesFromPeptideListBuilder.generate_peptides_df()

        num_records = df.index.size
        num_seqs = len(set(df['peptide']))

        decoy_df = df[df['annotation'].str.contains('Reverse')]
        num_decoy_records = decoy_df.index.size
        percent_decoy_records = (num_decoy_records/num_records * 100)
        num_decoy_seqs = len(set(decoy_df['peptide']))
        percent_decoy_seqs = (num_decoy_seqs/num_seqs * 100)



        df = df[~df['annotation'].str.contains('Reverse')]
        num_forward_recs = df.index.size
        num_forward_seqs = len(set(df['peptide']))


        msg = 'Records: Tot {}, Forward {} , Decoy {} ({:.0f}%)'.format(num_records, 
                                                                   num_forward_recs, 
                                                                   num_decoy_records, 
                                                                   percent_decoy_records)
        print(msg)

        msg = 'Sequences: Tot {}, Forward {}, Decoy {} ({:.0f}%)'.format(num_seqs, 
                                                                          num_forward_seqs, 
                                                                          num_decoy_seqs, 
                                                                          percent_decoy_seqs)
        print(msg)


        df['RatioRank'] = df["file_AREA_MEDIAN_RATIO_1"].rank(ascending=True)

        ax = sns.regplot(x="RatioRank", y="file_AREA_MEDIAN_RATIO_1", data=df)
        plt.figure()

        intersection = set(df['peptide']) & set(decoy_df['peptide'])
        print('Peptide sequences seen in both the decoy and forward sets {}'.format(intersection))

        cols = ['decoy','file_AREA_MEDIAN_RATIO_1', 'peptide', 'mod_locs', 'xcorr']
        return df[cols]
    
    
        
        
def get_comparisons():

    file_path = '{}/{}'.format(study_dir, 'comparisons.txt')
    df = pd.read_table(file_path)

    comparisons = df.apply(lambda x : Comparison(comparisonID=x['Comparison'], 
                                                 nibr_input=x['NIBR_INPUT'], 
                                                 ucb_input=x['UCB_INPUT'], 
                                                 ucb_output=x['UCB_OUTPUT'], 
                                                 desc=x, 
                                                 fdr=x['FDR']), 
                           axis=1)


    #for comp in comparisons:
    #    display(comp.desc)
    return comparisons

    print('DONE')
    
comparisons = get_comparisons()

def peptide_equals(pep1, pep2):
    pep1 = re.sub('[IL]', '[IL]', pep1)
    return bool(re.match(pep1, pep2))

In [ ]:
comp = comparisons[1]


In [ ]:
def clean_df(df):
    print('Cleaning up DF')
    cols = set(df.columns.values)

    # cols.remove('FEATURE')
    first_cols = ['IN_BOTH', 'Modifications_best_positions', 'QvalityPEP']
    cols = list(cols - set(first_cols))
    cols.sort()
    
    for col in first_cols:
        cols.insert(0, cols)
    
    #df.columns = map(str.upper, df.columns)
    df = df[cols]
    return df


In [ ]:
class SimplePeptide:
    
    def __init__(self, sequence):
        self.sequence = sequence
        self.pattern = re.sub('[IL]', '[IL]', sequence)
        self.myhash = hash(self.pattern)

    def __eq__(self, other):
        """Overrides the default implementation"""
        if isinstance(self, other.__class__):
            return bool(self.pattern == other.pattern)
        return False
    
    def __hash__(self):
        return self.myhash
    
    def __lt__(self, other):
        return self.pattern < other.pattern
    
    def __str__(self):
        return self.sequence

def do_sequence_overlap_bymerge(ip2DF=None,  pdDF=None, comp=None):
    ip2DF = ip2DF.copy()
    pdDF = pdDF.copy()

    print('Creating simple peps')
    ip2DF['peptide_ip2'] = ip2DF['peptide'].apply(lambda x: SimplePeptide(x))
    pdDF['peptide_pd'] = pdDF['Sequence'].apply(lambda x: SimplePeptide(x))
    
    print('Merging dfs')
    df = pd.merge(ip2DF, pdDF, left_on='peptide_ip2', right_on='peptide_pd', suffixes=['_ip2', '_pd'], how='outer')
    df['IN_BOTH'] = (df['peptide_ip2'] == df['peptide_pd'])
    
    data = {}

    print('Counting')
    ip2_peps = set(df['peptide_ip2'])
    pd_peps = set(df['peptide_pd'])
        
    pd_peps_with_ratios = set(df[~df.ABUNDANCE_RATIOS.isnull()].Sequence)

    intersection = ip2_peps & pd_peps
    percent_ip2_covered = len(intersection)/len(ip2_peps)*100

    data['IP2 Peps'] = len(ip2_peps) 
    data['PD Peps'] = len(pd_peps)
    data['Common Peps'] = len(intersection)
    data['% IP2 Covered in PD'] = percent_ip2_covered

    intersection_with_ratios = ip2_peps & pd_peps_with_ratios
    percent_ip2_covered = len(intersection_with_ratios)/len(ip2_peps)*100
    overlapping_pd_peps_missing_ratios = intersection - intersection_with_ratios 

    
    
    data['Quant PD Peps'] = len(pd_peps_with_ratios)
    data['Quant Common Peps'] = len(intersection_with_ratios)
    data['% IP2 Covered with PD Quant Peps'] = percent_ip2_covered

    correlations = do_ratio_plot(df, comp=comp, groupby_col='peptide_ip2')
    
    
    for key in correlations.keys():
        data[key] = correlations[key]
        data[key] = correlations[key]

    
    return (df, pd.Series(data))

def do_probe_overlap_bymerge(ip2DF=None,  pdDF=None, comp=None):
    ip2DF = ip2DF.copy()
    pdDF = pdDF.copy()
    
    # Remove non-probed peptides
    
    #pdDF = pdDF[pdDF['PROBE_MOD_LOCS'].apply(lambda x : len(x) > 0)]
    
    pdDF['PROBE_MOD_LOCS'] = pdDF['PROBE_MOD_LOCS'].apply(lambda x : re.sub(r"[\[\]'\s]", '', x).split(','))
    
    
    ip2DF['probe_peptide_ip2'] = ip2DF['peptide'] + '_C' + ip2DF.mod_locs.apply(lambda x : ',C'.join(map(str, x)))
    pdDF['probe_peptide_pd'] = pdDF['Sequence'] + '_' + pdDF.PROBE_MOD_LOCS.apply(lambda x : ','.join(x))

    df = pd.merge(ip2DF, pdDF, left_on='probe_peptide_ip2', right_on='probe_peptide_pd', suffixes=['_ip2', '_pd'], how='outer')
    df['IN_BOTH'] = (df['probe_peptide_ip2'] == df['probe_peptide_pd'])

    data = {}
    
    ip2_peps = set(df['probe_peptide_ip2'])
    pd_peps = set(df['probe_peptide_pd'])
    intersection = ip2_peps & pd_peps
    percent_ip2_covered = len(intersection)/len(ip2_peps)*100

    
    data['Probed IP2 Peps'] = len(ip2_peps) 
    data['Probed PD Peps'] = len(pd_peps)
    data['Probed Common Peps'] = len(intersection)
    data['% IP2 Covered in Probed PD'] = percent_ip2_covered

    
    pd_peps_with_ratios = set(df[~df['ABUNDANCE_RATIOS'].isnull()]['probe_peptide_pd'])
    intersection_with_ratios = ip2_peps & pd_peps_with_ratios
    percent_ip2_covered = len(intersection_with_ratios)/len(ip2_peps)*100

    
    overlapping_pd_peps_missing_ratios = intersection - intersection_with_ratios 
    
    data['Quant PD Probed Peps'] = len(pd_peps_with_ratios)
    data['Quant Common Probed Peps'] = len(intersection_with_ratios)
    data['% IP2 Covered with PD Quant Probed Peps'] = percent_ip2_covered
    
    correlations = do_ratio_plot(df, comp=comp, groupby_col='probe_peptide_ip2')
    print('Returned {}'.format(correlations))
    for key in correlations.keys():
        data[key] = correlations[key]
        data[key] = correlations[key]
        

    return (df, pd.Series(data))
     
 
def do_ratio_plot(df, comp=None, groupby_col=None):
    df = df.copy()
    pd_ratio_cols = ['Heavy/Light R', 'Light/Heavy R']

    ip2_ratio_col = 'file_AREA_MEDIAN_RATIO_1'
    
    #print(df.columns)
    
    correlations = {}
    for pd_ratio_col in pd_ratio_cols:
        tmpDF = df[df['IN_BOTH']] 
        tmpDF = df[~df[pd_ratio_col].isnull()] 
        
        groups = tmpDF.groupby(groupby_col)

        ip2_ratios = groups[ip2_ratio_col].median()
        pd_ratios = groups[pd_ratio_col].median()

        tmpDF = pd.DataFrame({ip2_ratio_col : ip2_ratios, pd_ratio_col : pd_ratios}).reset_index()

        # This has an R of 2.683 in PD. And here too
        #display(tmpDF[tmpDF[groupby_col].str.contains('LVAFCPFASSQVALENANAVSEGVVHEDLR')]) 
        #display(tmpDF)
        
        #display(tmpDF[tmpDF[ip2_ratio_col]>50]) 
        
        speamanCorr = spearmanr(tmpDF[ip2_ratio_col], tmpDF[pd_ratio_col])
        correlations['{} Spearman Corr'.format(pd_ratio_col)] = speamanCorr[0]
        correlations['{} Spearman PVal'.format(pd_ratio_col)] = speamanCorr[1]
        
        pearsonCorr = pearsonr(tmpDF[ip2_ratio_col], tmpDF[pd_ratio_col])
        correlations['{} Pearson Corr'.format(pd_ratio_col)] = pearsonCorr[0]
        correlations['{} Pearson PVal'.format(pd_ratio_col)] = pearsonCorr[1]

        
        #assert not np.isnan(correlations[pd_ratio_col][0]), 'Did not get a value back'

        g = sns.jointplot(x=ip2_ratio_col, y = pd_ratio_col, data=tmpDF, kind="reg",
                          color="r", size=7)

        g.fig.suptitle('PD Ratio: {}, Type: {}, Comprison:{} [{}], Spearman {}, Pearson {}'.format(pd_ratio_col, 
                                                                                      groupby_col, 
                                                                                      comp.comparisonID, 
                                                                                      comp.Description, 
                                                                                      speamanCorr[0], 
                                                                                      pearsonCorr[0]))

        plt.figure()
        
    return correlations

    

In [ ]:
results = {}
for comp in comparisons:
    if comp.get_pd_reader():
        print(comp.comparisonID)
        pdDF = comp.get_nibr_output()

        ip2DF = comp.get_ucb_input()
        
        parts = [comp.desc]
        
        result = do_sequence_overlap_bymerge(ip2DF=ip2DF, pdDF=pdDF, comp=comp)
        parts.append(result[1])
        
        file_path = '{}/{}'.format(results_dir, '{}_pep_overlap.txt'.format(comp.comparisonID))

        result[0].to_csv(file_path, sep='\t', index=False)
        
        result = do_probe_overlap_bymerge(ip2DF=ip2DF, pdDF=pdDF, comp=comp)
        parts.append(result[1])
        
        
        file_path = '{}/{}'.format(results_dir, '{}_pep_probe_overlap.txt'.format(comp.comparisonID))
        result[0].to_csv(file_path, sep='\t', index=False)

        results[comp.comparisonID] = parts

        """super_dict = {}
        for d in parts:
            for k, v in d.iteritems():  # d.items() in Python 3+
                super_dict[k] = v
                
        results.append(pd.Series(data=super_dict))"""    
    else:
        pass
        #print('Missing PD data for {}'.format(comp.desc))
        
print('Done doing comparisons')
#results = pd.concat(results, axis=1).T

#df = results
#df = clean_df(df)

#file_path = '{}/{}'.format(results_dir, 'peptide_compare.txt')

#df.to_csv(file_path, sep='\t', index=False)
#print('Writing files')

parts = []
for comp in results.keys():
    parts.append(pd.concat(results[comp]))

df = pd.concat(parts, axis=1).T

display(df)
#display(df.columns)

file_path = '{}/{}'.format(results_dir, 'peptide_compare.txt')

df.to_csv(file_path, sep='\t', index=False)
print('Done')

In [ ]:
def compare_to_target_psms(comp=None):
    print(comp.Description)
    ip2_pep_col = 'peptide_ip2'
    pd_pep_col = 'peptide_pd'
    ip2DF = comp.get_ucb_input()
    target_psmsDF = comp.get_pd_reader().get_target_psms()
    
    
    ip2DF[ip2_pep_col] = ip2DF['peptide']
    target_psmsDF[pd_pep_col] = target_psmsDF['Sequence']
    
    mergedDF = pd.merge(ip2DF, target_psmsDF, left_on=ip2_pep_col, right_on=pd_pep_col, how='outer')
    
    ip2_peps = set(mergedDF[ip2_pep_col])
    pd_peps = set(mergedDF[pd_pep_col])
    common = ip2_peps & pd_peps
    
    msg = '#IP2: {}, #PDTargetPSMs: {}, Common: {}'.format(len(ip2_peps), len(pd_peps), len(common))
    print(msg)
    
#comp = comparisons[1]   

#compare_to_target_psms(comparisons[1])

for comp in comparisons:
    if comp.get_pd_reader():
        compare_to_target_psms(comp)

In [ ]:
df.T

In [ ]:
## Compare older (SemiTryptic) with new
full_tryp_df = pd.read_csv('/home/jonesmic/gBuild/jonesmic_github/proteomics-scripts/datanocommit/03072018_proteomics_informatics_tc/KEA_EN80_UCB_RERUN/peptideList.csv')
full_tryp_df['SEQUENCE'] = full_tryp_df.sequence.str.replace('[()\d.]+','')
display(full_tryp_df[full_tryp_df['SEQUENCE'] == 'AAQDFFSTCR'])

# Note: This is also KEA semi trip but the latest is the one in KEA_EN80_12132017
semi_trip_df = pd.read_csv('/home/jonesmic/gBuild/jonesmic_github/proteomics-scripts/datanocommit/peptideList.csv')

semi_trip_df = pd.read_csv('/home/jonesmic/gBuild/jonesmic_github/proteomics-scripts/datanocommit/KEA_EN80_12132017/peptideList.csv')
semi_trip_df['SEQUENCE'] = semi_trip_df.sequence.str.replace('[()\d.]+','')

In [ ]:
mergeDF = pd.merge(full_tryp_df, semi_trip_df, on='sequence', suffixes=['_new', '_old'])
mergeDF = mergeDF[~(mergeDF['AREA_MEDIAN_RATIO_1_new'].isnull() | mergeDF['AREA_MEDIAN_RATIO_1_old'].isnull())]

g = sns.jointplot(x='AREA_MEDIAN_RATIO_1_new', y= 'AREA_MEDIAN_RATIO_1_old', data=mergeDF, kind="reg",
                  color="r", size=7)

g.fig.suptitle('New vs. Old KDE')

In [ ]:
comp = comparisons[1]
df = comp.get_ucb_input()
df.columns

In [ ]:
peptidesFromPeptideListBuilder = PeptidesFromPeptideListBuilder(comp.ucb_input)
df = peptidesFromPeptideListBuilder.generate_peptides_df()

decoy_df = df[df['annotation'].str.contains('Reverse')]

decoy_df.annotation